In [8]:
import pandas as pd
from modeling import feature_engineering, load_object, model_predict, save_csv,CB_target_encoding,LOO_target_encoding,label_encoder
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [9]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
training='training'
label='interest_integer'
knn_model=KNeighborsClassifier(n_neighbors=50,n_jobs=-1)
steps = [('scaler', StandardScaler()), ('KNN', knn_model)]
knn = Pipeline(steps)
clf=knn
df,columns,good_encoders,dp_table,df_tokenizer=load_object(clf.__class__.__name__+'_knn_thesis')


In [11]:
import functools
import optuna
from hashlib import md5
from optuna.samplers import TPESampler

def objective(df,columns,training,label,encoder_dict,df_test_listingID,df_train_listingID,trial):
    params = {
       
        'n_neighbors':trial.suggest_int('n_neighbors', 200, 500),
        'p': trial.suggest_int('p', 1,2),
        'weights': trial.suggest_categorical('weights', ['uniform', 'distance'])
    }
    knn_model=KNeighborsClassifier(n_jobs=-1,**params)
    steps = [('scaler', StandardScaler()), ('KNN', knn_model)]
    knn = Pipeline(steps)
    clf=knn
    test_pred,yoof,lloss=model_predict(df[columns],df[training],df[label], clf ,encoder_dict)
    model_id=md5(str(params).encode()).hexdigest()[:6]
    model_id='knn2'+clf.__class__.__name__+'_'+model_id+'_'+str(round(lloss,3))
    trial.set_user_attr('model_id',model_id)
    save_csv (model_id,test_pred,yoof,clf,df_test_listingID,df_train_listingID)
    return lloss


In [4]:
n=50
sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,good_encoders,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



[I 2020-05-30 12:33:03,166] Finished trial#0 with value: 0.6220252029265175 with parameters: {'n_neighbors': 173, 'p': 2, 'weights': 'distance'}. Best is trial#0 with value: 0.6220252029265175.
[I 2020-05-30 12:36:54,542] Finished trial#1 with value: 0.6208259028358645 with parameters: {'n_neighbors': 193, 'p': 2, 'weights': 'distance'}. Best is trial#1 with value: 0.6208259028358645.
[I 2020-05-30 12:40:42,716] Finished trial#2 with value: 0.6209748174141184 with parameters: {'n_neighbors': 196, 'p': 2, 'weights': 'distance'}. Best is trial#1 with value: 0.6208259028358645.
[I 2020-05-30 12:44:37,255] Finished trial#3 with value: 0.8642412088306096 with parameters: {'n_neighbors': 22, 'p': 1, 'weights': 'uniform'}. Best is trial#1 with value: 0.6208259028358645.
[I 2020-05-30 12:48:37,058] Finished trial#4 with value: 0.6303968172549065 with parameters: {'n_neighbors': 88, 'p': 1, 'weights': 'uniform'}. Best is trial#1 with value: 0.6208259028358645.
[I 2020-05-30 12:52:47,782] Finish

In [12]:
n=50
sampler=TPESampler(seed=0)
study = optuna.create_study(direction='minimize',sampler=sampler)
fn=functools.partial(objective,df,columns,training,label,good_encoders,df_test['listing_id'],df_train['listing_id'])
study.optimize(fn, n_trials=n,show_progress_bar=True)

/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



[I 2020-05-31 14:36:15,397] Finished trial#0 with value: 0.6257080170272372 with parameters: {'n_neighbors': 372, 'p': 2, 'weights': 'distance'}. Best is trial#0 with value: 0.6257080170272372.
[I 2020-05-31 14:40:10,424] Finished trial#1 with value: 0.626266379599636 with parameters: {'n_neighbors': 392, 'p': 2, 'weights': 'distance'}. Best is trial#0 with value: 0.6257080170272372.
[I 2020-05-31 14:44:12,103] Finished trial#2 with value: 0.6263590638064269 with parameters: {'n_neighbors': 395, 'p': 2, 'weights': 'distance'}. Best is trial#0 with value: 0.6257080170272372.
[I 2020-05-31 14:48:19,513] Finished trial#3 with value: 0.6308435887514422 with parameters: {'n_neighbors': 411, 'p': 2, 'weights': 'uniform'}. Best is trial#0 with value: 0.6257080170272372.
[I 2020-05-31 14:52:31,615] Finished trial#4 with value: 0.6328424954380342 with parameters: {'n_neighbors': 492, 'p': 2, 'weights': 'uniform'}. Best is trial#0 with value: 0.6257080170272372.
[I 2020-05-31 14:56:48,642] Finis

In [13]:
x=[{**t.params,**t.user_attrs} for t in study.get_trials()]
y=[ t.value for t in study.get_trials()]
table=pd.DataFrame(x)
table['score']=y
table.to_csv(clf.__class__.__name__+'_KNN1_grid_results.csv')

In [7]:
table.sort_values(by=['score'])

,n_neighbors,p,weights,model_id,score
41,198,1,distance,knnPipeline_536ad3_0.607,0.61
17,199,1,distance,knnPipeline_15e49c_0.607,0.61
31,199,1,distance,knnPipeline_15e49c_0.607,0.61
48,180,1,distance,knnPipeline_90919b_0.607,0.61
15,170,1,distance,knnPipeline_10ea59_0.607,0.61
28,200,1,distance,knnPipeline_b23dad_0.607,0.61
32,200,1,distance,knnPipeline_b23dad_0.607,0.61
34,200,1,distance,knnPipeline_b23dad_0.607,0.61
30,200,1,distance,knnPipeline_b23dad_0.607,0.61
42,200,1,distance,knnPipeline_b23dad_0.607,0.61
